# **TMBD API Extraction**

_John Andrew Dixon_

---

##### **Imports**

In [2]:
import json, os, time
import pandas as pd
import tmdbsimple as tmdb

##### **Helper Functions**

In [5]:
def get_movie_with_rating(movie_id):
    """Using the TMDB API, retrieves movie info and rating based on a given movie_id"""

    # Using the TMDB API, get movie based on "movie_id"
    movie = tmdb.Movies(movie_id)

    # Save movie info to a variable
    info = movie.info()
    # Save movie releases to a variable
    releases = movie.releases()

    # Loop through all releases for each country
    for country in releases["countries"]:
        # If the country is US, add the certification to the info dictionary
        if country["iso_3166_1"] == "US":
            info["certification"] = country["certification"]

    # Return the info dictionary
    return info

In [10]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename, "r+") as file:
        # Load existing data as a dictionary
        file_data = json.load(file)

        # Choose to extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)

        # Set file's current position at offset
        file.seek(0)

        # Convert back to JSON
        json.dump(file_data, file)

##### **Data Load**

In [12]:
# Load the previous "basics" file from part 1
basics_df = pd.read_csv("Data/title_basics.csv.gz")
# Verify it loaded
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


---

## **TMBD API Extraction**

In [ ]:
# Specify the folder where data will be saved
FOLDER = "Data/"
# Create the folder; if it exists, move on
os.makedirs(FOLDER, exist_ok=True)
# List folder's contents
os.listdir(FOLDER)

In [3]:
# Load API Key
with open("/Users/johna/.secret/tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [4]:
# Link API Key with API
tmdb.API_KEY = login["api-key"]

In [ ]:
# Specify years to get
YEARS_TO_GET = [2000, 2001]
# Create empty list to hold potential errors
errors = []

In [6]:
test = get_movie_with_rating("tt0848228")
test

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 166.244,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [7]:
test_ids = ["tt0848228", "tt0115937","tt0332280"]
results = []

for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e:
        errors.append((movie_id, e))
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28527,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10294,PG-13


In [8]:
errors

[('tt0115937',
  requests.exceptions.HTTPError('404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/tt0115937?api_key=48b4a37e995f85ebabd7bd2be6b4d687'))]